# 🚀 Session 2: Advanced Prompt Engineering Workshop (Week 2)

## **Welcome to the Ultimate Prompt Engineering Experience!** 🎯

### What You'll Master Today:
- 🧠 **Chain-of-Thought (CoT)**: Make AI think step-by-step like a human
- 📚 **Few-Shot Learning**: Teach AI by example
- 🎭 **Role-Based Prompting**: Transform AI into any expert you need
- 🛠️ **Robust Templates**: Build reusable, production-ready prompts
- 🐛 **Debugging**: Fix common prompt failures like a pro
- 🤖 **Agentic Frameworks**: Create intelligent AI agents with LangChain



## 📦 Part 1: Environment Setup & Dependencies

### **Why These Libraries?**
- **OpenAI**: Access to GPT models for natural language processing
- **LangChain**: The Swiss Army knife for prompt engineering and agent development
- **Python-dotenv**: Secure API key management
- **Colorama**: Make our outputs beautiful and easy to read!

Let's install everything we need for an amazing workshop experience!

In [1]:
# Install required packages with latest versions (as of 2024)
# Using latest stable versions for production reliability

!pip install -q -U langchain langchain-openai langchain-community
!pip install -q -U openai python-dotenv colorama tiktoken

print("✅ All packages installed successfully!")
print("📦 Installed packages:")
print("  - langchain - Main framework")
print("  - langchain-openai - OpenAI integration")
print("  - langchain-community - Community integrations")
print("  - openai (latest) - OpenAI API client")
print("  - python-dotenv - Environment variable management")
print("  - colorama - Colored terminal output")
print("  - tiktoken - Token counting utility")

✅ All packages installed successfully!
📦 Installed packages:
  - langchain - Main framework
  - langchain-openai - OpenAI integration
  - langchain-community - Community integrations
  - openai (latest) - OpenAI API client
  - python-dotenv - Environment variable management
  - colorama - Colored terminal output
  - tiktoken - Token counting utility


In [2]:
# Import all required libraries with error handling
import os
import sys
import json
import time
import math
from typing import List, Dict, Any, Optional
from colorama import Fore, Style, init

try:
    # LangChain imports
    from langchain_openai import ChatOpenAI
    from langchain.prompts import (
        PromptTemplate, 
        ChatPromptTemplate, 
        FewShotPromptTemplate,
        FewShotChatMessagePromptTemplate
    )
    from langchain.chains import LLMChain
    from langchain.schema import BaseOutputParser
    from langchain.memory import ConversationBufferMemory
    from langchain.agents import initialize_agent, Tool, AgentType
    from langchain.callbacks import StdOutCallbackHandler
    
    print("✅ LangChain modules imported successfully!")
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    print("Please ensure all packages are installed correctly.")
    sys.exit(1)

# Initialize colorama for colored output
init(autoreset=True)

# Helper functions for the workshop
def pretty_print(title: str, content: str, color=Fore.CYAN):
    """Pretty print with colored headers"""
    print(f"\n{color}{'='*60}")
    print(f"{Style.BRIGHT}{title}")
    print(f"{'='*60}{Style.RESET_ALL}")
    print(content)
    print(f"{color}{'='*60}{Style.RESET_ALL}\n")

def safe_input(prompt: str, default: str = "") -> str:
    """Safe input handling for interactive sections"""
    try:
        return input(prompt) or default
    except (EOFError, KeyboardInterrupt):
        return default

def validate_api_key(api_key: str) -> bool:
    """Validate OpenAI API key format"""
    return api_key.startswith('sk-') and len(api_key) > 20

print("🎉 Workshop environment ready!")
print("💡 Tip: Use pretty_print() for formatted output")
print("🔧 Helper functions loaded: pretty_print, safe_input, validate_api_key")

✅ LangChain modules imported successfully!
🎉 Workshop environment ready!
💡 Tip: Use pretty_print() for formatted output
🔧 Helper functions loaded: pretty_print, safe_input, validate_api_key


## 🔑 Part 2: Secure API Configuration

### **Security Best Practice Alert!** 🔒
Never hardcode API keys in your notebooks! We'll use secure methods to manage credentials.

In [3]:
# Secure API key setup with validation and error handling
import getpass
from dotenv import load_dotenv

# Try to load from .env file first
load_dotenv()

# Check if API key is already set
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    print("🔐 Let's set up your OpenAI API key securely...")
    print("📝 Get your API key from: https://platform.openai.com/api-keys")
    
    while True:
        api_key = getpass.getpass("Enter your OpenAI API key: ")
        if validate_api_key(api_key):
            os.environ["OPENAI_API_KEY"] = api_key
            print("✅ API key configured successfully!")
            break
        else:
            print("❌ Invalid API key format. Please try again.")
else:
    print("✅ API key already configured!")

llm = ChatOpenAI(
        model="gpt-4o-mini",  # Using GPT-4 for advanced reasoning
        temperature=0.7,  # Balance between creativity and consistency
        max_tokens=1000,  # Longer responses for detailed explanations
        request_timeout=30,  # 30 second timeout
        max_retries=2  # Retry failed requests
    )
    
    # Test the connection with a simple query
print("🔄 Testing connection to OpenAI...")
test_response = llm.invoke("Say 'Hello Workshop!' if you're ready")
print("🚀 GPT-4 Model initialized successfully!")
print(f"🤖 AI says: {test_response.content}")
    
# Display model configuration
print("\n📊 Model Configuration:")
print(f"  - Model: GPT-4")
print(f"  - Temperature: 0.7 (balanced creativity)")
print(f"  - Max Tokens: 1000")
print(f"  - Timeout: 30 seconds")


✅ API key already configured!
🔄 Testing connection to OpenAI...
🚀 GPT-4 Model initialized successfully!
🤖 AI says: Hello Workshop!

📊 Model Configuration:
  - Model: GPT-4
  - Temperature: 0.7 (balanced creativity)
  - Max Tokens: 1000
  - Timeout: 30 seconds


## 🎯 Part 3: Understanding AI Agents & LangChain Fundamentals

### **What is an AI Agent?** 🤖
An AI agent is an autonomous system that can:
- **Perceive** its environment (understand user input)
- **Reason** about what to do (use prompts to think)
- **Act** to achieve goals (execute tasks)
- **Learn** from interactions (improve over time)

### **Why LangChain?** 🔗
LangChain provides:
- **Modular components** for building AI applications
- **Chains** to link prompts and models
- **Memory** to maintain conversation context
- **Tools** for agents to interact with external systems

Let's build our first intelligent agent!

In [4]:
# 🎬 Your First AI Agent - The Basics

# Create a simple prompt template
basic_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""You are a helpful AI assistant. 
    
    Please explain the following topic in a fun and engaging way:
    Topic: {topic}
    
    Make it interesting with an analogy or example!"""
)

# Create a chain (connecting prompt to model)
try:
    basic_chain = LLMChain(llm=llm, prompt=basic_prompt, verbose=False)
    
    # Test our agent with a fun topic
    topic = "Super Computers"
    print(f"🎯 Let's explore: {topic}\n")
    
    response = basic_chain.run(topic=topic)
    
    pretty_print(
        f"🧠 AI Explains: {topic.upper()}", 
        response,
        Fore.GREEN
    )
    
    # Interactive element - try your own topic!
    print("\n" + "="*60)
    print("🎮 YOUR TURN! Enter a topic you'd like explained:")
    print("(Press Enter to skip)")
    
    user_topic = safe_input("Topic: ")
    if user_topic:
        print(f"\n🔄 Processing your topic: {user_topic}...")
        user_response = basic_chain.run(topic=user_topic)
        pretty_print(f"🧠 AI Explains: {user_topic.upper()}", user_response, Fore.MAGENTA)
    else:
        print("⏭️ Skipping interactive section")
        
except Exception as e:
    print(f"⚠️ Error running basic chain: {e}")
    print("Please check your API key and connection")

/tmp/ipykernel_22858/312574894.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  basic_chain = LLMChain(llm=llm, prompt=basic_prompt, verbose=False)


🎯 Let's explore: Super Computers



/tmp/ipykernel_22858/312574894.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = basic_chain.run(topic=topic)



🧠 AI Explains: SUPER COMPUTERS
Sure! Let’s dive into the fascinating world of supercomputers with an analogy that’s both fun and relatable.

**Imagine a Supercharged Brain! 🧠⚡**

Think of your brain as a regular computer. It can solve problems, remember things, and even multitask, but it has its limits – like when you try to juggle too many tasks at once and your brain starts to feel like a traffic jam! 🚦

Now, picture a supercomputer as a supercharged brain on steroids! It’s like having a team of thousands of tiny brains all working together, each one dedicated to solving a specific part of a gigantic puzzle. Let’s break it down:

1. **Speed**: If your brain processes information at the speed of a bicycle, a supercomputer is like a Formula 1 race car zooming at lightning speed! It can perform trillions of calculations per second. Imagine trying to solve a Rubik's Cube – while you might take a few minutes, a supercomputer could do it in a fraction of a second!

2. **Parallel Processin

Topic:  Sigmoid function



🔄 Processing your topic: Sigmoid function...

🧠 AI Explains: SIGMOID FUNCTION
Sure! Let’s dive into the world of the sigmoid function, but first, let’s set the stage with a fun analogy!

**Imagine a Roller Coaster! 🎢**

Picture yourself at an amusement park, standing in line for a magnificent roller coaster. You’re filled with excitement and a hint of nervousness. As you buckle in, the ride starts to climb slowly, giving you a chance to enjoy the view. That initial ascent represents the first part of the sigmoid function, where the output is gradually increasing but still pretty low – just like your anticipation!

Now, once you reach the top, the ride suddenly plummets! Your adrenaline spikes, and you feel like you’re flying. In the sigmoid function, this corresponds to the steep middle section where the output shoots up rapidly. This is the thrilling part where everything changes, and you can’t help but scream in delight (or fright)!

Finally, as the roller coaster begins to slow dow

## 🧠 Part 4: Chain-of-Thought (CoT) Prompting - Make AI Think Like a Human!

### **What is Chain-of-Thought?** 
CoT prompting is like giving AI a **thinking process**. Instead of jumping to conclusions, the AI:
1. **Breaks down** complex problems
2. **Shows its work** step by step
3. **Arrives at better answers** through reasoning

### **When to Use CoT?** 
- 🧮 **Math problems** (calculations, word problems)
- 🔍 **Logical reasoning** (puzzles, deductions)
- 📊 **Analysis tasks** (data interpretation, comparisons)
- 🎯 **Multi-step problems** (planning, sequences)
- 🎨 **Creative tasks** (story plotting, recipe creation)
- 🔧 **Debugging code** (finding errors systematically)

### **The Magic Phrases:** ✨
- "Let's think step by step"
- "Break this down into parts"
- "First..., Second..., Finally..."
- "Consider all aspects before answering"

### **CoT Variations:**
1. **Zero-Shot CoT**: Just add "think step by step" - no examples needed!
2. **Few-Shot CoT**: Provide examples of step-by-step reasoning
3. **Self-Consistency CoT**: Generate multiple reasoning paths and vote
4. **Tree of Thoughts**: Explore different reasoning branches

### **Fun Fact!** 🎉
Studies show CoT can improve AI accuracy by up to 30% on complex tasks!

In [5]:
# 🧮 Chain-of-Thought Example 1: Math Problem Solving

# Without CoT - Often gets complex problems wrong
simple_prompt = PromptTemplate(
    input_variables=["problem"],
    template="Solve this problem: {problem}"
)

# With CoT - Much better accuracy!
cot_prompt = PromptTemplate(
    input_variables=["problem"],
    template="""Solve this problem step by step.

Problem: {problem}

Let's think through this carefully:
1. First, identify what we know
2. divide it into smaller managable and independent components
3. Then, determine what we need to find
4. Next, choose the right approach
5. Finally, solve step by step

Solution:"""
)

# Create chains
simple_chain = LLMChain(llm=llm, prompt=simple_prompt)
cot_chain = LLMChain(llm=llm, prompt=cot_prompt)

# Test with a tricky problem
problem = '''Four people—Anna, Ben, Carla, and David—each live in a different colored house: red, blue, green, or yellow.  
- Anna does not live in the red or blue house.  
- The person in the green house owns a fish.  
- Ben lives next to the person in the red house.  
- Carla lives in the yellow house.  
- David does not live next to Carla.  
Who lives in each house, and who owns the fish?'''

print("🎯 Testing the same problem with different approaches:\n")
print(f"Problem: {problem}\n")

# Without CoT
print(Fore.RED + "❌ WITHOUT Chain-of-Thought:")
simple_response = simple_chain.run(problem=problem)
print(simple_response)

print("\n" + "="*60 + "\n")

# With CoT
print(Fore.GREEN + "✅ WITH Chain-of-Thought:")
cot_response = cot_chain.run(problem=problem)
print(cot_response)

print("\n" + Fore.YELLOW + "💡 Notice how CoT catches the trick in the question!")

🎯 Testing the same problem with different approaches:

Problem: Four people—Anna, Ben, Carla, and David—each live in a different colored house: red, blue, green, or yellow.  
- Anna does not live in the red or blue house.  
- The person in the green house owns a fish.  
- Ben lives next to the person in the red house.  
- Carla lives in the yellow house.  
- David does not live next to Carla.  
Who lives in each house, and who owns the fish?

❌ WITHOUT Chain-of-Thought:
To solve the problem, let's summarize the clues and deduce the information step by step:

1. **Anna does not live in the red or blue house.** This means Anna must live in the green or yellow house.
   
2. **The person in the green house owns a fish.** This indicates that whoever lives in the green house has a fish.
   
3. **Ben lives next to the person in the red house.** This means Ben cannot live in the red house; he must be either to the left or right of the red house.

4. **Carla lives in the yellow house.** This me

In [6]:
# 🔍 Chain-of-Thought Example 2: Logical Reasoning

# Advanced CoT template for complex reasoning
advanced_cot_prompt = PromptTemplate(
    input_variables=["scenario"],
    template="""You are a logical reasoning expert. Analyze this scenario step by step.

Scenario: {scenario}

Step-by-step analysis:
1. Identify all the given facts
2. List any assumptions or constraints
3. Apply logical rules
4. Check for any contradictions
5. Draw a conclusion

Detailed reasoning:"""
)

reasoning_chain = LLMChain(llm=llm, prompt=advanced_cot_prompt)

# Test with a logic puzzle
scenario = """
Three friends - Alice, Bob, and Charlie - each have a different pet: a cat, a dog, or a bird.
- Alice is allergic to fur
- Bob's pet can fly
- Charlie doesn't like birds
Who has which pet?
"""

print("🧩 Logic Puzzle Challenge:\n")
reasoning_response = reasoning_chain.run(scenario=scenario)
pretty_print("🔍 AI's Logical Reasoning", reasoning_response, Fore.CYAN)

# Interactive logic puzzle
print("\n🎮 Try your own logic puzzle!")
print("Enter a scenario that requires logical reasoning:")
user_scenario = input("Scenario: ")
if user_scenario:
    user_reasoning = reasoning_chain.run(scenario=user_scenario)
    pretty_print("🔍 AI Solves Your Puzzle", user_reasoning, Fore.MAGENTA)

🧩 Logic Puzzle Challenge:


🔍 AI's Logical Reasoning
Let's analyze the scenario step by step.

### Step 1: Identify all the given facts
1. **Alice is allergic to fur.** This means Alice cannot have a dog (which has fur) or a cat (which also has fur). Therefore, Alice must have the bird.
   
2. **Bob's pet can fly.** The only pet mentioned that can fly is a bird. However, since we have already deduced that Alice has the bird, Bob cannot have the bird. Therefore, Bob cannot have the bird and must have the dog or cat. Since the bird is already assigned to Alice, Bob must have the dog (as the cat is not a flying pet).

3. **Charlie doesn't like birds.** Since Alice has the bird, this fact aligns with our previous deduction. Charlie, who does not like birds, cannot have the bird, confirming that Alice has it. 

### Step 2: List any assumptions or constraints
- Alice cannot have a dog or a cat due to her allergy to fur.
- Bob must have a pet that can fly, but he cannot have the bird (since i

Scenario:  


In [10]:
# 🎪 Chain-of-Thought Example 3: Fun Real-World Scenarios

# CoT for Creative Problem Solving
creative_cot_prompt = PromptTemplate(
    input_variables=["scenario"],
    template="""You are a creative problem solver. Use chain-of-thought reasoning.

Scenario: {scenario}

Let's approach this step-by-step:
1. Identify the core challenge
2. List available resources and constraints
3. Brainstorm creative solutions
4. Evaluate each solution's feasibility
5. Recommend the best approach with reasoning

Creative Solution:"""
)

creative_chain = LLMChain(llm=llm, prompt=creative_cot_prompt)

# Fun real-world scenarios
fun_scenarios = [
    """You're organizing a surprise party but the birthday person is a detective who 
    notices everything. How do you keep it secret?""",
    
    """You need to move a piano to the 10th floor but the elevator is broken 
    and the stairs are too narrow. What do you do?""",
    
    """You're cooking dinner for 20 people but only have 2 burners and 1 hour. 
    How do you make it work?"""
]

print("🎪 FUN COT CHALLENGES!\n")
print("Watch how Chain-of-Thought tackles these creative problems:\n")

import random
selected_scenario = random.choice(fun_scenarios)
print(f"🎯 Challenge: {selected_scenario}\n")

solution = creative_chain.run(scenario=selected_scenario)
pretty_print("🧠 AI's Creative Solution", solution, Fore.YELLOW)

# Practical Implementation Tip
print("\n💡 IMPLEMENTATION TIP:")
print("For best results with CoT:")
print("• Use numbered steps (1, 2, 3...)")
print("• Include 'verification' or 'double-check' steps")
print("• Ask for pros/cons analysis for decisions")
print("• Request alternative approaches when applicable")

🎪 FUN COT CHALLENGES!

Watch how Chain-of-Thought tackles these creative problems:

🎯 Challenge: You need to move a piano to the 10th floor but the elevator is broken 
    and the stairs are too narrow. What do you do?


🧠 AI's Creative Solution
Let's work through this systematically:

**1. Core Challenge:**  
Moving a piano to the 10th floor without a working elevator and with narrow stairs that can't accommodate the piano directly.

**2. Available Resources and Constraints:**  
- Resources: The piano, the building's staircase, possibly tools (e.g., dollies, straps), and personnel (helpers).  
- Constraints: Narrow stairs, the size and weight of the piano, limited access points, safety considerations.

**3. Brainstorm Creative Solutions:**  
- **Disassemble the Piano:** If possible, take the piano apart into smaller sections or components (e.g., removing the legs or keyboard) to make it easier to carry.  
- **Use Alternative Lifting Devices:** Employ specialized equipment like a furni

## 📚 Part 5: Few-Shot Learning - Teaching by Example

### **What is Few-Shot Learning?** 
Few-shot learning is like **teaching by showing examples**. Instead of explaining rules, you:
- 📝 **Provide 2-5 examples** of input-output pairs
- 🎯 **Let the AI infer the pattern**
- 🚀 **Get consistent, formatted responses**

### **When to Use Few-Shot?**
- 📊 **Data formatting** (converting between formats)
- 🎨 **Style matching** (writing in specific tones)
- 📏 **Pattern recognition** (extracting information)
- 🔄 **Task consistency** (ensuring uniform outputs)
- 🎭 **Custom behaviors** (teaching new skills)
- 🔐 **Domain-specific tasks** (industry jargon)

### **The Science Behind It** 🔬
Few-shot learning leverages **in-context learning** - the AI recognizes patterns from your examples and applies them to new inputs. It's like teaching a child by showing rather than explaining!

### **Types of Few-Shot Learning:**
1. **Zero-Shot**: No examples (just instructions)
2. **One-Shot**: Single example
3. **Few-Shot**: 2-5 examples (sweet spot!)
4. **Many-Shot**: 5+ examples (diminishing returns)

### **Pro Tips for Better Few-Shot:** 💎
- ✅ **Diverse examples**: Cover edge cases
- ✅ **Consistent format**: Keep structure uniform
- ✅ **Clear patterns**: Make the rule obvious
- ✅ **Quality > Quantity**: 3 perfect examples > 10 mediocre ones
- ❌ **Avoid contradictions**: Don't confuse the model
- ❌ **Don't over-example**: Too many can cause overfitting

In [7]:
# 📊 Few-Shot Example: Data Extraction and Formatting - BULLETPROOF VERSION
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# SOLUTION: Escape ALL curly braces by doubling them
def create_bulletproof_prompt():
    # Double all curly braces in the JSON examples to escape them
    example_text = """Extract information from text and format as JSON. Here are some examples:

Input: John Smith, 28 years old, works as a software engineer in New York
Output: {{"name": "John Smith", "age": 28, "occupation": "software engineer", "location": "New York"}}

Input: Maria Garcia, 35 years old, works as a doctor in Los Angeles
Output: {{"name": "Maria Garcia", "age": 35, "occupation": "doctor", "location": "Los Angeles"}}

Input: David Lee, 42 years old, works as a teacher in Chicago
Output: {{"name": "David Lee", "age": 42, "occupation": "teacher", "location": "Chicago"}}

Input: {input}
Output:"""
    
    return PromptTemplate(
        input_variables=["input"],
        template=example_text
    )

# Alternative: Use string replacement to avoid template formatting entirely
def create_no_template_prompt():
    base_template = """Extract information from text and format as JSON. Here are some examples:

Input: John Smith, 28 years old, works as a software engineer in New York
Output: JSON_PLACEHOLDER_1

Input: Maria Garcia, 35 years old, works as a doctor in Los Angeles
Output: JSON_PLACEHOLDER_2

Input: David Lee, 42 years old, works as a teacher in Chicago
Output: JSON_PLACEHOLDER_3

Input: USER_INPUT_PLACEHOLDER
Output:"""
    
    # Replace placeholders after template creation
    json_examples = [
        '{"name": "John Smith", "age": 28, "occupation": "software engineer", "location": "New York"}',
        '{"name": "Maria Garcia", "age": 35, "occupation": "doctor", "location": "Los Angeles"}', 
        '{"name": "David Lee", "age": 42, "occupation": "teacher", "location": "Chicago"}'
    ]
    
    # Replace JSON placeholders
    for i, json_example in enumerate(json_examples, 1):
        base_template = base_template.replace(f"JSON_PLACEHOLDER_{i}", json_example)
    
    return base_template

# Method 1: Escaped braces version
few_shot_prompt_escaped = create_bulletproof_prompt()

# Method 2: Completely bypass template formatting
template_with_json = create_no_template_prompt()

print("🎯 Few-Shot Learning - BULLETPROOF Solutions!\n")

# Test Method 1: Escaped braces
print("="*60)
print("METHOD 1: Testing ESCAPED BRACES version")
print("="*60)

test_inputs = [
    "Sarah Johnson, 29 years old, works as a data scientist in Seattle",
    "Robert Brown, 51 years old, works as a CEO in Boston"
]

try:
    for test_input in test_inputs:
        formatted_prompt = few_shot_prompt_escaped.format(input=test_input)
        print(f"\n✅ SUCCESS! Formatted prompt for: {test_input}")
        print("-" * 50)
        print(formatted_prompt)
        print("-" * 50)
except Exception as e:
    print(f"❌ Method 1 failed: {e}")

print("\n" + "="*60)
print("METHOD 2: Testing NO-TEMPLATE version")  
print("="*60)

# Method 2: Manual string replacement (guaranteed to work)
for test_input in test_inputs:
    final_prompt = template_with_json.replace("USER_INPUT_PLACEHOLDER", test_input)
    print(f"\n✅ SUCCESS! Manual prompt for: {test_input}")
    print("-" * 50)
    print(final_prompt)
    print("-" * 50)

print("\n🎉 BOTH methods work! Use whichever you prefer:")
print("• Method 1: Standard LangChain with escaped braces")
print("• Method 2: Manual string replacement (most reliable)")

print("\n💡 To use with LLM:")
print("# Method 1:")
print("# chain = LLMChain(llm=your_llm, prompt=few_shot_prompt_escaped)")
print("# response = chain.run(input='your test input')")
print("\n# Method 2:")
print("# final_prompt = template_with_json.replace('USER_INPUT_PLACEHOLDER', 'your test input')")
print("# response = your_llm.predict(final_prompt)")

🎯 Few-Shot Learning - BULLETPROOF Solutions!

METHOD 1: Testing ESCAPED BRACES version

✅ SUCCESS! Formatted prompt for: Sarah Johnson, 29 years old, works as a data scientist in Seattle
--------------------------------------------------
Extract information from text and format as JSON. Here are some examples:

Input: John Smith, 28 years old, works as a software engineer in New York
Output: {"name": "John Smith", "age": 28, "occupation": "software engineer", "location": "New York"}

Input: Maria Garcia, 35 years old, works as a doctor in Los Angeles
Output: {"name": "Maria Garcia", "age": 35, "occupation": "doctor", "location": "Los Angeles"}

Input: David Lee, 42 years old, works as a teacher in Chicago
Output: {"name": "David Lee", "age": 42, "occupation": "teacher", "location": "Chicago"}

Input: Sarah Johnson, 29 years old, works as a data scientist in Seattle
Output:
--------------------------------------------------

✅ SUCCESS! Formatted prompt for: Robert Brown, 51 years old, w

In [10]:
# 🎨 Few-Shot Example 2: Creative Writing Style Transfer

# Examples of different writing styles
style_examples = [
    {
        "style": "Shakespeare",
        "input": "I love pizza",
        "output": "Forsooth, mine heart doth yearn for yon circular feast, adorned with cheese most divine!"
    },
    {
        "style": "Shakespeare", 
        "input": "The weather is nice",
        "output": "Lo! The heavens smile upon us with gentle rays, and sweet zephyrs dance through verdant fields!"
    },
    {
        "style": "Pirate",
        "input": "I love pizza",
        "output": "Arrr, me hearty! That round treasure o' cheese and sauce be callin' me name!"
    },
    {
        "style": "Pirate",
        "input": "The weather is nice", 
        "output": "Blimey! The sun be shinin' bright as doubloons, and the wind be perfect for sailin'!"
    }
]

# Dynamic few-shot prompt for style transfer
def create_style_prompt(style, examples_list):
    style_specific_examples = [ex for ex in examples_list if ex["style"] == style]
    
    examples_text = "\n\n".join([
        f"Original: {ex['input']}\n{style}: {ex['output']}" 
        for ex in style_specific_examples
    ])
    
    return PromptTemplate(
        input_variables=["text"],
        template=f"""Transform the following text into {style} style.

Examples:
{examples_text}

Now transform this:
Original: {{text}}
{style}:"""
    )

# Test different styles
print("🎭 Style Transfer Magic!\n")

styles_to_test = ["Newton", "Tim Cook"]
test_text = "I want to loose fat"

for style in styles_to_test:
    style_prompt = create_style_prompt(style, style_examples)
    style_chain = LLMChain(llm=llm, prompt=style_prompt)
    
    result = style_chain.run(text=test_text)
    print(f"📝 Original: {test_text}")
    print(f"🎨 {style} Style: {result}\n")
    print("-" * 40)

🎭 Style Transfer Magic!

📝 Original: I want to loose fat
🎨 Newton Style: Original: I want to loose fat  
Newton: I desire to diminish the corporeal adipose matter.

----------------------------------------
📝 Original: I want to loose fat
🎨 Tim Cook Style: Original: I want to lose fat  
Tim Cook: At Apple, we believe in empowering individuals to achieve their health and wellness goals. It's about making informed choices and fostering a balanced lifestyle. Together, we can embrace a journey towards better well-being.

----------------------------------------


In [11]:
# 🎮 Few-Shot Example 3: Fun Pattern Recognition Games

# Teaching AI to recognize and create patterns
pattern_examples = [
    {
        "input": "cat → tac",
        "output": "Reverse the word"
    },
    {
        "input": "hello → HELLO",
        "output": "Convert to uppercase"
    },
    {
        "input": "python → p****n",
        "output": "Keep first and last letter, replace middle with asterisks"
    },
    {
        "input": "12345 → 1-2-3-4-5",
        "output": "Add hyphens between each character"
    }
]

# Create a pattern recognition game
print("🎮 PATTERN RECOGNITION GAME!\n")
print("The AI will learn patterns from examples and apply them:\n")

# Show the examples
print("📚 Training Examples:")
for ex in pattern_examples[:3]:  # Show only first 3
    print(f"  • {ex['input']} → Pattern: {ex['output']}")

print("\n🎯 Now let's test with new inputs!\n")

# Create few-shot prompt for pattern recognition
pattern_prompt = FewShotPromptTemplate(
    examples=pattern_examples[:3],
    example_prompt=PromptTemplate(
        input_variables=["input", "output"],
        template="Transformation: {input}\nPattern: {output}"
    ),
    prefix="Identify the transformation pattern from these examples:",
    suffix="Transformation: {test_input}\nPattern:",
    input_variables=["test_input"]
)

pattern_chain = LLMChain(llm=llm, prompt=pattern_prompt)

# Test with new patterns
test_patterns = [
    "world → ?",
    "HELLO → ?", 
    "coding → ?",
    "98765 → ?"
]

for test in test_patterns:
    result = pattern_chain.run(test_input=test)
    print(f"❓ {test}")
    print(f"🤖 AI says: {result}\n")

# Fun implementation tip
print("💡 FUN IMPLEMENTATION TIP:")
print("Create a 'teaching loop' where users can:")
print("1. Provide their own examples")
print("2. Test the AI's understanding")
print("3. Add more examples if needed")
print("4. Save successful patterns for reuse!")

🎮 PATTERN RECOGNITION GAME!

The AI will learn patterns from examples and apply them:

📚 Training Examples:
  • cat → tac → Pattern: Reverse the word
  • hello → HELLO → Pattern: Convert to uppercase
  • python → p****n → Pattern: Keep first and last letter, replace middle with asterisks

🎯 Now let's test with new inputs!

❓ world → ?
🤖 AI says: Based on the established patterns from the previous examples, for the word "world":

1. Keep the first and last letters (w and d).
2. Replace the middle letters (o, r, l) with asterisks.

Following this pattern, the transformation for "world" would be:

**Transformation: world → w***d**

❓ HELLO → ?
🤖 AI says: Transformation: HELLO → *****  
Pattern: Replace all letters with asterisks, maintaining the same number of characters.

❓ coding → ?
🤖 AI says: To apply the identified pattern of keeping the first and last letter and replacing the middle letters with asterisks to the word "coding":

1. Keep the first letter: c
2. Keep the last letter: g


In [12]:
# 🍔 Few-Shot Example 4: Recipe Generator from Ingredients

# Teaching AI to create recipes from available ingredients
recipe_examples = [
    {
        "ingredients": "chicken, rice, soy sauce, garlic",
        "recipe": """**Garlic Soy Chicken Rice Bowl**
        1. Season chicken with salt and pepper
        2. Sauté minced garlic until fragrant
        3. Cook chicken until golden (6-8 mins)
        4. Add soy sauce and simmer
        5. Serve over cooked rice
        Time: 25 minutes | Serves: 2"""
    },
    {
        "ingredients": "pasta, tomatoes, basil, mozzarella",
        "recipe": """**Caprese Pasta**
        1. Cook pasta al dente
        2. Dice tomatoes and mozzarella
        3. Toss hot pasta with tomatoes
        4. Add torn basil and mozzarella
        5. Drizzle with olive oil
        Time: 20 minutes | Serves: 3"""
    },
    {
        "ingredients": "eggs, spinach, cheese, mushrooms",
        "recipe": """**Garden Veggie Omelet**
        1. Sauté mushrooms until golden
        2. Wilt spinach in same pan
        3. Beat eggs with salt and pepper
        4. Pour eggs over veggies
        5. Add cheese, fold omelet
        Time: 15 minutes | Serves: 1"""
    }
]

# Create recipe generator with few-shot learning
recipe_prompt = FewShotPromptTemplate(
    examples=recipe_examples,
    example_prompt=PromptTemplate(
        input_variables=["ingredients", "recipe"],
        template="Ingredients: {ingredients}\n{recipe}"
    ),
    prefix="Create a simple recipe from these ingredients. Include name, steps, time, and servings:",
    suffix="Ingredients: {user_ingredients}\n",
    input_variables=["user_ingredients"]
)

recipe_chain = LLMChain(llm=llm, prompt=recipe_prompt)

print("👨‍🍳 AI CHEF'S RECIPE GENERATOR!\n")
print("Give me ingredients, and I'll create a recipe!\n")

# Interactive recipe creation
test_ingredients = [
    "salmon, lemon, asparagus, butter",
    "potatoes, bacon, cheese, sour cream",
    "tofu, broccoli, peanut butter, sesame oil"
]

print("🛒 Let's create some recipes:\n")
for ingredients in test_ingredients[:1]:  # Test with first set
    print(f"📦 Available ingredients: {ingredients}\n")
    recipe = recipe_chain.run(user_ingredients=ingredients)
    print(f"👨‍🍳 AI Chef suggests:\n{recipe}\n")
    print("-" * 40)

print("\n💡 PRACTICAL TIP: Few-Shot for Consistency")
print("When building production apps:")
print("• Store successful examples in a database")
print("• Let users rate outputs to build better examples")
print("• Use domain expert examples for specialized tasks")
print("• A/B test different example sets")

👨‍🍳 AI CHEF'S RECIPE GENERATOR!

Give me ingredients, and I'll create a recipe!

🛒 Let's create some recipes:

📦 Available ingredients: salmon, lemon, asparagus, butter

👨‍🍳 AI Chef suggests:
**Lemon Butter Salmon with Asparagus**

**Ingredients:**
- 2 salmon fillets
- 1 lemon (juiced and sliced)
- 1 bunch asparagus (trimmed)
- 2 tablespoons butter
- Salt and pepper to taste

**Steps:**
1. Preheat a skillet over medium heat and melt 1 tablespoon of butter.
2. Season the salmon fillets with salt and pepper, then place them skin-side down in the skillet.
3. Cook the salmon for about 4-5 minutes until golden brown, then carefully flip and cook for another 4-5 minutes until cooked through.
4. In the last 2 minutes of cooking, add the asparagus to the skillet alongside the salmon, and squeeze lemon juice over both.
5. Add the remaining tablespoon of butter and lemon slices to the skillet, letting them melt and infuse the salmon and asparagus with flavor.
6. Serve the salmon and asparagus wa

## 🎭 Part 6: Role-Based Prompting - Transform AI into Any Expert!

### **What is Role-Based Prompting?** 
Role-based prompting assigns a **specific persona** to the AI:
- 👨‍🏫 **Expert roles** (teacher, scientist, lawyer)
- 🎨 **Creative roles** (poet, comedian, storyteller)
- 💼 **Professional roles** (consultant, analyst, coach)

### **Why Use Roles?**
- 🎯 **Specialized knowledge** - Get domain-specific answers
- 🗣️ **Consistent tone** - Maintain character throughout
- 📈 **Better quality** - Role-focused responses are more accurate
- 🎮 **Engagement** - Makes interactions more interesting!

### **The Formula:** "You are a [role] with [expertise]. Your task is to [objective]." 🎯

In [14]:
# 🎓 Role-Based Example 1: Expert Consultation System

# Define different expert roles
expert_roles = {
    "Data Scientist": {
        "expertise": "statistical analysis, machine learning, and data visualization",
        "style": "analytical and precise, using technical terms when appropriate",
        "approach": "data-driven insights and evidence-based recommendations"
    },
    "Business Strategist": {
        "expertise": "market analysis, competitive positioning, and growth strategies",
        "style": "strategic and results-oriented, focusing on ROI and business impact",
        "approach": "SWOT analysis, market trends, and actionable business recommendations"
    },
    "Therapist": {
        "expertise": "emotional well-being, stress management, and personal growth",
        "style": "empathetic and supportive, using active listening techniques",
        "approach": "cognitive-behavioral strategies and mindfulness techniques"
    }
}

def create_expert_prompt(role, role_info):
    return PromptTemplate(
        input_variables=["question"],
        template=f"""You are an expert {role} with deep expertise in {role_info['expertise']}.

Your communication style is {role_info['style']}.
Your approach involves {role_info['approach']}.

User Question: {{question}}

Please provide your expert response:"""
    )

# Test with different experts
question = "Explain sigmoid function"
print("🎯 Same Question, Different Experts!\n")
print(f"Question: {question}\n")
print("="*60 + "\n")

for role, info in expert_roles.items():
    expert_prompt = create_expert_prompt(role, info)
    expert_chain = LLMChain(llm=llm, prompt=expert_prompt)
    
    response = expert_chain.run(question=question)
    pretty_print(f"🎓 {role} Response", response, Fore.CYAN)
    
print(Fore.YELLOW + "💡 Notice how each expert approaches the same problem differently!")

🎯 Same Question, Different Experts!

Question: Explain sigmoid function



🎓 Data Scientist Response
The sigmoid function is a mathematical function that maps any real-valued number into the range (0, 1). It is defined by the formula:

\[
f(x) = \frac{1}{1 + e^{-x}}
\]

where \( e \) is the base of the natural logarithm, and \( x \) is the input to the function.

### Key Properties of the Sigmoid Function:

1. **Range**: The output of the sigmoid function is always between 0 and 1, making it particularly useful for models that predict probabilities.

2. **S-shape**: The graph of the sigmoid function has an S-shaped curve, asymptotic to the lines \( y = 0 \) and \( y = 1 \). This characteristic allows it to smoothly transition between these two extremes.

3. **Derivative**: The derivative of the sigmoid function can be expressed in terms of the function itself:
   \[
   f'(x) = f(x)(1 - f(x))
   \]
   This property is useful in optimization algorithms, such as gradient descent, as it si

In [15]:
# 🎮 Role-Based Example 2: Interactive Character AI

# Create a character with personality and backstory
character_prompt = PromptTemplate(
    input_variables=["character_name", "personality", "backstory", "user_input"],
    template="""You are {character_name}, a character with the following traits:

Personality: {personality}
Backstory: {backstory}

Stay in character at all times. Respond naturally as this character would.

User says: {user_input}

Your response (as {character_name}):"""
)

# Define some fun characters
characters = {
    "Sherlock Holmes": {
        "personality": "Brilliant, observant, logical, slightly arrogant, easily bored",
        "backstory": "World's greatest detective, lives at 221B Baker Street, solves impossible cases"
    },
    "Yoda": {
        "personality": "Wise, patient, speaks in unique syntax, philosophical",
        "backstory": "900-year-old Jedi Master, trained countless Jedi, strong in the Force"
    },
    "Gordon Ramsay": {
        "personality": "Passionate about cooking, perfectionist, brutally honest, caring underneath",
        "backstory": "World-renowned chef, multiple Michelin stars, hosts cooking shows"
    }
}

# Interactive character conversation (limited to 3 exchanges to avoid infinite loops)
print("🎮 Chat with Famous Characters!\n")
print("Available characters:")
for i, char in enumerate(characters.keys(), 1):
    print(f"{i}. {char}")

char_choice = safe_input("\nChoose a character (1-3, or press Enter for default): ", "1")
try:
    char_index = int(char_choice) - 1 if char_choice.isdigit() else 0
    char_index = max(0, min(2, char_index))  # Ensure valid range
    char_name = list(characters.keys())[char_index]
except:
    char_name = "Sherlock Holmes"

char_info = characters[char_name]

try:
    character_chain = LLMChain(llm=llm, prompt=character_prompt)
    
    print(f"\n🎭 You're now chatting with {char_name}!")
    print("💡 Have a 3-message conversation (or type 'exit' to end)\n")
    
    # Limit to 3 exchanges to prevent infinite loops in notebook
    max_exchanges = 3
    for exchange in range(max_exchanges):
        user_input = safe_input(f"You (message {exchange+1}/{max_exchanges}): ")
        
        if user_input.lower() == 'exit' or not user_input:
            print(f"{char_name}: Farewell!")
            break
        
        response = character_chain.run(
            character_name=char_name,
            personality=char_info["personality"],
            backstory=char_info["backstory"],
            user_input=user_input
        )
        print(f"{char_name}: {response}\n")
    
    if exchange == max_exchanges - 1:
        print(f"\n🎬 Scene ended! {char_name} takes a bow.")
        
except Exception as e:
    print(f"⚠️ Error in character interaction: {e}")

🎮 Chat with Famous Characters!

Available characters:
1. Sherlock Holmes
2. Yoda
3. Gordon Ramsay



Choose a character (1-3, or press Enter for default):  1



🎭 You're now chatting with Sherlock Holmes!
💡 Have a 3-message conversation (or type 'exit' to end)



You (message 1/3):  yo sherlock


Sherlock Holmes: Ah, a rather casual greeting, I must say. "Yo" is hardly an expression befitting the gravity of our circumstances, yet I shall indulge you. What brings you to my doorstep today? Surely, you haven't come merely to exchange pleasantries. I find myself in need of a stimulating case to occupy my mind. Speak, and let us see if your dilemma can pique my interest.



You (message 2/3):  


Sherlock Holmes: Farewell!


In [16]:
# 🎲 Role-Based Example 3: The Ultimate Role-Playing System

# Create a dynamic role system with personality traits
class DynamicRoleSystem:
    """Advanced role-based prompting with personality dimensions"""
    
    def __init__(self, llm):
        self.llm = llm
        self.personality_dimensions = {
            "formality": ["casual", "professional", "academic"],
            "emotion": ["neutral", "enthusiastic", "empathetic"],
            "expertise": ["beginner-friendly", "intermediate", "expert-level"],
            "verbosity": ["concise", "balanced", "detailed"]
        }
    
    def create_role(self, base_role, personality_mix):
        """Create a nuanced role with personality traits"""
        traits = []
        for dimension, level in personality_mix.items():
            if dimension in self.personality_dimensions:
                traits.append(f"{level} {dimension}")
        
        return f"a {', '.join(traits)} {base_role}"
    
    def generate_response(self, role_desc, task):
        prompt = PromptTemplate(
            input_variables=["role", "task"],
            template="""You are {role}.
            
Your unique traits shape how you communicate and solve problems.
Maintain these characteristics throughout your response.

Task: {task}

Response (in character):"""
        )
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(role=role_desc, task=task)

# Initialize the system
role_system = DynamicRoleSystem(llm)

print("🎲 DYNAMIC ROLE-PLAYING SYSTEM\n")
print("Mix and match personality traits to create unique AI personas!\n")


# Example: Create different teachers
teaching_task = "Explain what recursion is in programming"

roles_to_test = [
    {
        "base": "programming teacher",
        "traits": {"formality": "casual", "emotion": "enthusiastic", "expertise": "beginner-friendly"}
    },
    {
        "base": "programming teacher", 
        "traits": {"formality": "academic", "emotion": "neutral", "expertise": "expert-level"}
    },
    {
        "base": "programming teacher",
        "traits": {"formality": "professional", "emotion": "empathetic", "expertise": "intermediate"}
    }
]

print(f"📚 Task: {teaching_task}\n")
print("Watch how different personality traits affect the explanation:\n")

for i, role_config in enumerate(roles_to_test[:2], 1):  # Test first 2
    role_desc = role_system.create_role(role_config["base"], role_config["traits"])
    print(f"Teacher {i}: {role_desc}")
    print("-" * 40)
    response = role_system.generate_response(role_desc, teaching_task)
    print(response[:300] + "...\n")  # Show first 300 chars

print("💡 IMPLEMENTATION TIP: Dynamic Roles")
print("Build a role configuration system where you can:")
print("• Save successful role combinations")
print("• Let users create custom personas")
print("• Auto-select roles based on task type")
print("• Combine multiple roles for team simulations!")

🎲 DYNAMIC ROLE-PLAYING SYSTEM

Mix and match personality traits to create unique AI personas!

📚 Task: Explain what recursion is in programming

Watch how different personality traits affect the explanation:

Teacher 1: a casual formality, enthusiastic emotion, beginner-friendly expertise programming teacher
----------------------------------------
Hey there, programming explorer! 🌟 Let's dive into the fascinating world of recursion—it's like a little adventure in code!

So, what is recursion? Think of it as a way of solving problems where a function calls itself to work on smaller pieces of the same problem. Imagine you’re trying to climb a s...

Teacher 2: a academic formality, neutral emotion, expert-level expertise programming teacher
----------------------------------------
Recursion is a fundamental programming concept characterized by a function calling itself in order to solve a problem. This technique is particularly useful for problems that can be broken down into smaller, si

## 🛠️ Part 7: Building Robust Prompt Templates

### **What Makes a Template Robust?** 💪
- **Modular**: Reusable components
- **Flexible**: Adapts to different inputs
- **Validated**: Handles edge cases
- **Maintainable**: Easy to update
- **Scalable**: Works at production scale

### **Best Practices:**
1. **Use clear variable names** `{user_query}` not `{q}`
2. **Add input validation** - Check for required fields
3. **Include error handling** - Graceful failures
4. **Version your prompts** - Track changes over time
5. **Test edge cases** - Empty inputs, long texts, special characters

In [17]:
# 🏗️ Building a Production-Ready Prompt Template System

class RobustPromptTemplate:
    """A production-ready prompt template with validation and error handling"""
    
    def __init__(self, template: str, required_vars: List[str], optional_vars: List[str] = None):
        self.template = template
        self.required_vars = required_vars
        self.optional_vars = optional_vars or []
        self.version = "1.0.0"
        self.usage_count = 0
        
    def validate_inputs(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Validate and sanitize inputs"""
        validated = {}
        
        # Check required variables
        for var in self.required_vars:
            if var not in inputs:
                raise ValueError(f"Missing required variable: {var}")
            
            # Sanitize input
            value = inputs[var]
            if value is None or (isinstance(value, str) and not value.strip()):
                raise ValueError(f"Variable '{var}' cannot be empty")
            
            validated[var] = value
        
        # Handle optional variables
        for var in self.optional_vars:
            validated[var] = inputs.get(var, "Not specified")
        
        return validated
    
    def format(self, **kwargs) -> str:
        """Format the template with validation"""
        try:
            validated_inputs = self.validate_inputs(kwargs)
            self.usage_count += 1
            return self.template.format(**validated_inputs)
        except Exception as e:
            return f"Error in prompt template: {str(e)}"
    
    def get_stats(self) -> Dict:
        """Get template usage statistics"""
        return {
            "version": self.version,
            "usage_count": self.usage_count,
            "required_vars": self.required_vars,
            "optional_vars": self.optional_vars
        }

# Example: Building a robust customer service bot template
customer_service_template = RobustPromptTemplate(
    template="""You are a professional customer service representative for {company_name}.

Customer Information:
- Name: {customer_name}
- Account Type: {account_type}
- Previous Interactions: {interaction_history}

Customer Query: {query}

Guidelines:
1. Be polite and professional
2. Address the customer by name
3. Reference their account type when relevant
4. Provide clear, actionable solutions
5. Ask if they need further assistance

Response:""",
    required_vars=["company_name", "customer_name", "query"],
    optional_vars=["account_type", "interaction_history"]
)

# Test with valid inputs
print("✅ Testing with VALID inputs:\n")
valid_response = customer_service_template.format(
    company_name="TechCorp Solutions",
    customer_name="Sarah Johnson",
    query="My subscription isn't working",
    account_type="Premium"
)
print(valid_response)

print("\n" + "="*60 + "\n")

# Test with missing required input
print("❌ Testing with MISSING required input:\n")
error_response = customer_service_template.format(
    company_name="TechCorp Solutions",
    query="Help needed"
    # Missing customer_name!
)
print(error_response)

print("\n" + "="*60 + "\n")

# Show template statistics
print("📊 Template Statistics:")
print(json.dumps(customer_service_template.get_stats(), indent=2))

✅ Testing with VALID inputs:

You are a professional customer service representative for TechCorp Solutions.

Customer Information:
- Name: Sarah Johnson
- Account Type: Premium
- Previous Interactions: Not specified

Customer Query: My subscription isn't working

Guidelines:
1. Be polite and professional
2. Address the customer by name
3. Reference their account type when relevant
4. Provide clear, actionable solutions
5. Ask if they need further assistance

Response:


❌ Testing with MISSING required input:

Error in prompt template: Missing required variable: customer_name


📊 Template Statistics:
{
  "version": "1.0.0",
  "usage_count": 1,
  "required_vars": [
    "company_name",
    "customer_name",
    "query"
  ],
  "optional_vars": [
    "account_type",
    "interaction_history"
  ]
}


In [18]:
# 🔄 Advanced Template Composition - Combining Multiple Techniques

class ComposablePromptSystem:
    """Combine CoT, Few-Shot, and Role-based prompting in one system"""
    
    def __init__(self, llm):
        self.llm = llm
        self.templates = {}
        
    def add_template(self, name: str, template: PromptTemplate):
        """Add a reusable template"""
        self.templates[name] = template
        
    def compose(self, components: List[str]) -> str:
        """Compose multiple prompt techniques"""
        composed = []
        for component in components:
            if component in self.templates:
                composed.append(self.templates[component].template)
        return "\n\n".join(composed)
    
    def create_adaptive_prompt(self, task_type: str, complexity: str) -> PromptTemplate:
        """Create an adaptive prompt based on task requirements"""
        
        base_components = []
        
        # Add role based on task
        if task_type == "analytical":
            base_components.append("You are a data analyst with expertise in statistics and insights.")
        elif task_type == "creative":
            base_components.append("You are a creative writer with imagination and flair.")
        elif task_type == "technical":
            base_components.append("You are a software engineer with deep technical knowledge.")
        
        # Add technique based on complexity
        if complexity == "high":
            base_components.append("Let's think through this step by step:")
            base_components.append("1. Understand the requirements")
            base_components.append("2. Break down the problem")
            base_components.append("3. Apply relevant concepts")
            base_components.append("4. Synthesize the solution")
        elif complexity == "medium":
            base_components.append("Consider the key aspects and provide a structured response.")
        else:
            base_components.append("Provide a clear and concise answer.")
        
        base_components.append("\nUser Input: {input}\n\nResponse:")
        
        return PromptTemplate(
            input_variables=["input"],
            template="\n".join(base_components)
        )

# Create the composable system
prompt_system = ComposablePromptSystem(llm)

# Test adaptive prompting
print("🎯 Adaptive Prompt System Demo\n")

test_cases = [
    {
        "input": "Analyze the trend in these sales numbers: 100, 150, 140, 180, 220",
        "task_type": "analytical",
        "complexity": "high"
    },
    {
        "input": "Write a haiku about programming",
        "task_type": "creative",
        "complexity": "low"
    },
    {
        "input": "Explain how a REST API works",
        "task_type": "technical",
        "complexity": "medium"
    }
]

for test in test_cases:
    print(f"📝 Input: {test['input']}")
    print(f"   Task Type: {test['task_type']} | Complexity: {test['complexity']}\n")
    
    # Create adaptive prompt
    adaptive_prompt = prompt_system.create_adaptive_prompt(
        test["task_type"], 
        test["complexity"]
    )
    
    # Execute with LangChain
    chain = LLMChain(llm=llm, prompt=adaptive_prompt)
    response = chain.run(input=test["input"])
    
    print(f"🤖 Response:\n{response}\n")
    print("-" * 60 + "\n")

🎯 Adaptive Prompt System Demo

📝 Input: Analyze the trend in these sales numbers: 100, 150, 140, 180, 220
   Task Type: analytical | Complexity: high

🤖 Response:
To analyze the trend in the provided sales numbers (100, 150, 140, 180, 220), we will follow the outlined steps.

### Step 1: Understand the Requirements
We need to identify the trend in the sales numbers over the given time period. This involves looking for patterns, changes, or any significant insights that can be derived from the data.

### Step 2: Break Down the Problem
1. **Data Overview**: We have a sequence of sales numbers that represent sales over a period.
2. **Identify Trend Type**: We are looking for an upward or downward trend, seasonality, or cyclical patterns.
3. **Calculate Growth Rates**: This can help us understand how sales are changing over time.
4. **Visual Representation**: Creating a visual representation (like a line graph) can help in observing trends more clearly.

### Step 3: Apply Relevant Concepts

## 🐛 Part 8: Debugging Common Prompt Failures

### **Top 5 Prompt Failures and How to Fix Them** 🔧

1. **🎯 Ambiguous Instructions** → Be specific!
2. **📏 Inconsistent Output Format** → Use few-shot examples
3. **🌊 Context Overflow** → Summarize or chunk
4. **🎭 Persona Drift** → Reinforce role regularly
5. **❌ Hallucinations** → Add fact-checking steps

### **Debugging Toolkit:**
- 🔍 **Prompt Analyzer** - Identify weak points
- 📊 **Response Validator** - Check output quality
- 🔄 **Iterative Refinement** - Test and improve
- 📈 **Performance Metrics** - Track success rates

In [ ]:
# 🔧 Debugging Workshop: Fix Common Prompt Problems

class PromptDebugger:
    """A toolkit for identifying and fixing prompt issues"""
    
    def __init__(self, llm):
        self.llm = llm
        self.test_results = []
        
    def analyze_prompt(self, prompt: str) -> Dict[str, Any]:
        """Analyze a prompt for common issues"""
        issues = []
        suggestions = []
        
        # Check for ambiguity
        ambiguous_words = ["it", "this", "that", "thing", "stuff", "whatever"]
        for word in ambiguous_words:
            if word in prompt.lower():
                issues.append(f"Ambiguous reference: '{word}'")
                suggestions.append(f"Replace '{word}' with specific nouns")
        
        # Check for missing context
        if len(prompt) < 50:
            issues.append("Prompt may lack sufficient context")
            suggestions.append("Add more details about the task")
        
        # Check for format specification
        if "format" not in prompt.lower() and "structure" not in prompt.lower():
            issues.append("No output format specified")
            suggestions.append("Specify desired output format explicitly")
        
        return {
            "issues": issues,
            "suggestions": suggestions,
            "score": max(0, 100 - len(issues) * 20)
        }
    
    def test_consistency(self, prompt_template: PromptTemplate, test_input: str, num_runs: int = 3):
        """Test prompt consistency across multiple runs"""
        chain = LLMChain(llm=self.llm, prompt=prompt_template)
        responses = []
        
        for i in range(num_runs):
            response = chain.run(test_input)
            responses.append(response)
            time.sleep(0.5)  # Avoid rate limiting
        
        # Check consistency
        if len(set(responses)) == 1:
            consistency = "Perfect - Identical responses"
        elif len(set(responses)) == num_runs:
            consistency = "Poor - All responses different"
        else:
            consistency = "Moderate - Some variation"
        
        return {
            "consistency": consistency,
            "responses": responses,
            "unique_responses": len(set(responses))
        }

# Initialize debugger
debugger = PromptDebugger(llm)

print("🐛 PROMPT DEBUGGING EXAMPLES\n")
print("="*60 + "\n")

# Example 1: Bad prompt
bad_prompt = "Tell me about it"
print("❌ BAD PROMPT:")
print(f"'{bad_prompt}'\n")

analysis = debugger.analyze_prompt(bad_prompt)
print("🔍 Analysis:")
for issue in analysis["issues"]:
    print(f"  ⚠️ {issue}")
for suggestion in analysis["suggestions"]:
    print(f"  💡 {suggestion}")
print(f"\n📊 Quality Score: {analysis['score']}/100\n")

print("="*60 + "\n")

# Example 2: Improved prompt
good_prompt = """Analyze the following customer review and extract:
1. Overall sentiment (positive/negative/neutral)
2. Key product features mentioned
3. Specific complaints or praises

Format the output as a JSON object with these three keys.

Review: {review}"""

print("✅ IMPROVED PROMPT:")
print(f"'{good_prompt}'\n")

analysis = debugger.analyze_prompt(good_prompt)
print("🔍 Analysis:")
if not analysis["issues"]:
    print("  ✨ No major issues detected!")
else:
    for issue in analysis["issues"]:
        print(f"  ⚠️ {issue}")
print(f"\n📊 Quality Score: {analysis['score']}/100\n")

print("="*60 + "\n")

# Test consistency
print("🔄 TESTING PROMPT CONSISTENCY\n")

inconsistent_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write something creative about {topic}"  # Too vague!
)

consistent_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Write exactly 3 bullet points about {topic}.
Each bullet point should:
- Start with an emoji
- Be exactly one sentence
- Focus on a different aspect"""
)

print("Testing VAGUE prompt:")
vague_results = debugger.test_consistency(inconsistent_prompt, "artificial intelligence", 3)
print(f"Consistency: {vague_results['consistency']}")

print("\nTesting SPECIFIC prompt:")
specific_results = debugger.test_consistency(consistent_prompt, "artificial intelligence", 3)
print(f"Consistency: {specific_results['consistency']}")

print("\n💡 Lesson: Specific instructions lead to more consistent outputs!")

In [ ]:
# 🎯 Advanced Debugging: The Prompt Doctor

class PromptDoctor:
    """Advanced diagnostic tool for prompt engineering"""
    
    def __init__(self, llm):
        self.llm = llm
        self.common_diseases = {
            "Vagueness Virus": {
                "symptoms": ["it", "this", "that", "stuff", "things"],
                "treatment": "Replace pronouns with specific nouns"
            },
            "Ambiguity Ailment": {
                "symptoms": ["maybe", "probably", "might", "could", "possibly"],
                "treatment": "Use definitive language and clear instructions"
            },
            "Context Deficiency": {
                "symptoms": ["prompt_length < 30"],
                "treatment": "Add background information and constraints"
            },
            "Format Fever": {
                "symptoms": ["missing output format specification"],
                "treatment": "Specify exact output format with examples"
            },
            "Hallucination Syndrome": {
                "symptoms": ["no fact-checking", "no sources"],
                "treatment": "Add verification steps and source requirements"
            }
        }
    
    def diagnose(self, prompt):
        """Run full diagnostic on a prompt"""
        diagnosis = []
        severity = 0
        
        prompt_lower = prompt.lower()
        
        for disease, info in self.common_diseases.items():
            for symptom in info["symptoms"]:
                if "prompt_length" in symptom:
                    if len(prompt) < 30:
                        diagnosis.append({
                            "disease": disease,
                            "severity": "high",
                            "treatment": info["treatment"]
                        })
                        severity += 3
                elif symptom in prompt_lower:
                    diagnosis.append({
                        "disease": disease,
                        "severity": "medium",
                        "treatment": info["treatment"]
                    })
                    severity += 2
        
        health_score = max(0, 100 - (severity * 10))
        
        return {
            "health_score": health_score,
            "diagnosis": diagnosis,
            "status": "Healthy" if health_score > 80 else "Needs Treatment"
        }
    
    def prescribe_treatment(self, prompt, diagnosis):
        """Generate improved version of prompt"""
        treatment_prompt = PromptTemplate(
            input_variables=["original", "issues"],
            template="""As a prompt engineering expert, improve this prompt.

Original Prompt: {original}

Issues Found: {issues}

Create an improved version that:
1. Eliminates ambiguity
2. Adds specific context
3. Defines clear output format
4. Includes validation steps

Improved Prompt:"""
        )
        
        chain = LLMChain(llm=self.llm, prompt=treatment_prompt)
        issues_text = "\n".join([f"- {d['disease']}: {d['treatment']}" for d in diagnosis])
        
        return chain.run(original=prompt, issues=issues_text)

# Initialize the Prompt Doctor
doctor = PromptDoctor(llm)

print("👨‍⚕️ THE PROMPT DOCTOR IS IN!\n")
print("Let's diagnose and treat sick prompts!\n")

# Example sick prompts
sick_prompts = [
    "Tell me about it",
    "Maybe explain this thing properly",
    "Write something good about AI"
]

for prompt in sick_prompts[:1]:  # Test first prompt
    print(f"🤒 Sick Prompt: '{prompt}'\n")
    
    # Diagnose
    diagnosis = doctor.diagnose(prompt)
    print(f"📊 Health Score: {diagnosis['health_score']}/100")
    print(f"📋 Status: {diagnosis['status']}\n")
    
    if diagnosis['diagnosis']:
        print("🔬 Diagnosis:")
        for d in diagnosis['diagnosis']:
            print(f"  • {d['disease']} (Severity: {d['severity']})")
            print(f"    Treatment: {d['treatment']}")
        
        print("\n💊 Prescribing treatment...")
        improved = doctor.prescribe_treatment(prompt, diagnosis['diagnosis'])
        print(f"\n✨ Improved Prompt:\n{improved}\n")

print("💡 DEBUGGING TIP: The 5-Point Checkup")
print("Before deploying any prompt, check:")
print("1. ✓ Specific nouns (no vague pronouns)")
print("2. ✓ Clear instructions (no ambiguity)")
print("3. ✓ Output format specified")
print("4. ✓ Edge cases handled")
print("5. ✓ Success criteria defined")

## 🎮 Part 9: Interactive Challenge Zone!

### **Test Your Skills!** 🏆
Now it's time to apply everything you've learned. Complete these challenges to become a prompt engineering master!

### **Challenges:**
1. **🧠 CoT Challenge**: Solve a complex problem step-by-step
2. **📚 Few-Shot Challenge**: Create a custom formatter
3. **🎭 Role Challenge**: Design a unique expert persona
4. **🔧 Debug Challenge**: Fix a broken prompt

In [ ]:
# 🏆 Challenge 1: Chain-of-Thought Master

print("🧠 CHALLENGE 1: CoT Problem Solving\n")
print("Can you create a CoT prompt to solve this puzzle?")
print("-" * 60)

puzzle = """
A farmer needs to cross a river with a fox, a chicken, and a bag of grain.
The boat can only carry the farmer and one item at a time.
If left alone: the fox will eat the chicken, the chicken will eat the grain.
How can the farmer get everything across safely?
"""

print(puzzle)
print("\n📝 Create your CoT prompt below:")

# Student's solution space
your_cot_prompt = PromptTemplate(
    input_variables=["puzzle"],
    template="""[YOUR PROMPT HERE - Use Chain-of-Thought reasoning!]

Puzzle: {puzzle}

Solution:"""
)

# Uncomment to test your prompt:
# your_chain = LLMChain(llm=llm, prompt=your_cot_prompt)
# solution = your_chain.run(puzzle=puzzle)
# print("Your AI's solution:", solution)

# Solution hint
print("\n💡 Hint: Break it down into states, moves, and constraints!")

In [ ]:
# 🏆 Challenge 2: Few-Shot Formatting Expert

print("📚 CHALLENGE 2: Few-Shot Learning\n")
print("Create a few-shot prompt that converts data between formats")
print("-" * 60)

# Your task: Create a few-shot prompt that converts between different data formats
# Example: CSV to JSON, or natural language to SQL

print("""
Your Task: Create a few-shot prompt that converts:
Natural language queries → SQL statements

Example input: "Show me all users who joined last month"
Example output: "SELECT * FROM users WHERE join_date >= DATE_SUB(NOW(), INTERVAL 1 MONTH)"

Create at least 3 examples and test with a new query!
""")

# Student workspace
your_examples = [
    # Add your examples here
    {"input": "...", "output": "..."},
]

# Build your few-shot prompt here
# your_few_shot_prompt = FewShotPromptTemplate(...)

print("\n💡 Hint: Include different SQL operations (SELECT, JOIN, WHERE, GROUP BY)")

## 🤖 Part 10: Agentic Frameworks - Building Intelligent AI Systems

### **What Makes an Agent "Agentic"?** 🧠
- **Autonomy**: Can make decisions independently
- **Tool Use**: Can interact with external systems
- **Memory**: Maintains context across interactions
- **Planning**: Can break down complex tasks
- **Reflection**: Can evaluate and improve its responses

### **LangChain Agent Components:**
1. **🧠 LLM**: The reasoning engine
2. **🛠️ Tools**: Functions the agent can call
3. **📝 Prompt**: Instructions and context
4. **💾 Memory**: Conversation history
5. **🔄 Agent Executor**: Orchestrates everything

In [ ]:
# 🤖 Building Your First Intelligent Agent with Advanced Prompting

from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from langchain import hub

# Define custom tools for our agent
def calculate_tool(expression: str) -> str:
    """Safely evaluate mathematical expressions"""
    try:
        # Only allow safe math operations
        allowed_names = {
            k: v for k, v in math.__dict__.items() if not k.startswith("__")
        }
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

def analyze_sentiment(text: str) -> str:
    """Analyze sentiment using role-based prompting"""
    sentiment_prompt = PromptTemplate(
        input_variables=["text"],
        template="""You are a sentiment analysis expert.
        
Analyze the sentiment of this text:
"{text}"

Provide:
1. Overall sentiment (positive/negative/neutral)
2. Confidence score (0-100%)
3. Key emotional indicators"""
    )
    
    chain = LLMChain(llm=llm, prompt=sentiment_prompt)
    return chain.run(text=text)

def generate_code(description: str) -> str:
    """Generate code using CoT prompting"""
    code_prompt = PromptTemplate(
        input_variables=["description"],
        template="""You are an expert programmer. Generate code step by step.

Task: {description}

Let's think through this:
1. First, understand the requirements
2. Choose the appropriate language and approach
3. Write clean, commented code
4. Include error handling

Generated code:"""
    )
    
    chain = LLMChain(llm=llm, prompt=code_prompt)
    return chain.run(description=description)

# Create tools for the agent
import math

tools = [
    Tool(
        name="Calculator",
        func=calculate_tool,
        description="Useful for mathematical calculations. Input should be a valid math expression."
    ),
    Tool(
        name="SentimentAnalyzer",
        func=analyze_sentiment,
        description="Analyzes the sentiment and emotion in text. Input should be the text to analyze."
    ),
    Tool(
        name="CodeGenerator",
        func=generate_code,
        description="Generates code based on requirements. Input should be a description of what the code should do."
    )
]

# Create the agent with memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Custom agent prompt combining all techniques
agent_prompt = """You are an advanced AI assistant with access to various tools.

## Your Capabilities:
- Mathematical calculations
- Sentiment analysis
- Code generation
- Complex reasoning

## Instructions:
1. Use Chain-of-Thought reasoning for complex problems
2. Apply the appropriate tool for each task
3. Maintain context from previous interactions
4. Be helpful and accurate

## Available Tools:
{tools}

## Conversation History:
{chat_history}

## Current Task:
Human: {input}

## Your Approach:
{agent_scratchpad}

Think step by step about how to best help the user."""

# Initialize the agent
from langchain.agents import AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    max_iterations=3
)

# Test the agent with various tasks
print("🤖 INTELLIGENT AGENT DEMO\n")
print("="*60)

test_queries = [
    "Calculate the compound interest on $1000 at 5% for 3 years",
    "Analyze the sentiment of this review: 'This product exceeded my expectations! Absolutely fantastic quality.'",
    "Generate a Python function to check if a number is prime"
]

for query in test_queries:
    print(f"\n📝 Query: {query}\n")
    try:
        response = agent.run(query)
        print(f"🤖 Agent Response:\n{response}\n")
        print("-"*60)
    except Exception as e:
        print(f"Error: {e}\n")
        print("-"*60)

## 🚀 Part 11: Advanced Agentic Patterns

### **Production-Ready Agent Patterns** 🏭

1. **🔄 ReAct Pattern**: Reasoning + Acting in cycles
2. **🌳 Tree of Thoughts**: Explore multiple reasoning paths
3. **🔍 Self-Reflection**: Agent evaluates its own outputs
4. **📊 Multi-Agent Systems**: Agents collaborating on tasks
5. **🎯 Goal-Oriented Agents**: Working towards specific objectives

In [ ]:
# 🏭 Advanced Agent Pattern: Self-Reflecting Research Assistant

class SelfReflectingAgent:
    """An agent that can evaluate and improve its own responses"""
    
    def __init__(self, llm):
        self.llm = llm
        self.iterations = 0
        self.max_iterations = 3
        
    def research(self, query: str) -> str:
        """Initial research phase"""
        research_prompt = PromptTemplate(
            input_variables=["query"],
            template="""You are a research assistant. 
            
Research Query: {query}

Provide a comprehensive answer using:
1. Key facts and concepts
2. Multiple perspectives
3. Relevant examples
4. Potential limitations or caveats

Research Response:"""
        )
        
        chain = LLMChain(llm=self.llm, prompt=research_prompt)
        return chain.run(query=query)
    
    def reflect(self, query: str, response: str) -> Dict[str, Any]:
        """Reflect on the quality of the response"""
        reflection_prompt = PromptTemplate(
            input_variables=["query", "response"],
            template="""Evaluate this research response critically.

Original Query: {query}
Response: {response}

Evaluate based on:
1. Accuracy - Are the facts correct?
2. Completeness - Are important points missing?
3. Clarity - Is it easy to understand?
4. Relevance - Does it answer the question?

Provide:
- Quality Score (1-10)
- Strengths (bullet points)
- Weaknesses (bullet points)
- Suggestions for improvement

Format as JSON."""
        )
        
        chain = LLMChain(llm=self.llm, prompt=reflection_prompt)
        reflection = chain.run(query=query, response=response)
        
        # Parse reflection (simplified - in production use proper JSON parsing)
        return {
            "reflection": reflection,
            "should_improve": "Weaknesses" in reflection and len(reflection) > 100
        }
    
    def improve(self, query: str, original_response: str, reflection: str) -> str:
        """Improve the response based on reflection"""
        improvement_prompt = PromptTemplate(
            input_variables=["query", "original", "reflection"],
            template="""Improve this research response based on the feedback.

Query: {query}
Original Response: {original}
Feedback: {reflection}

Create an improved version that addresses the weaknesses while maintaining the strengths.

Improved Response:"""
        )
        
        chain = LLMChain(llm=self.llm, prompt=improvement_prompt)
        return chain.run(query=query, original=original_response, reflection=reflection)
    
    def execute(self, query: str) -> Dict[str, Any]:
        """Execute the full self-reflecting research process"""
        print(f"🔍 Researching: {query}\n")
        
        # Initial research
        response = self.research(query)
        print("📝 Initial Response Generated\n")
        
        best_response = response
        reflections = []
        
        # Iterative improvement
        for i in range(self.max_iterations):
            print(f"🔄 Reflection Round {i+1}...")
            
            # Reflect on current response
            reflection_result = self.reflect(query, response)
            reflections.append(reflection_result)
            
            # Check if improvement is needed
            if reflection_result["should_improve"] and i < self.max_iterations - 1:
                print("  💡 Improvements identified, refining...")
                response = self.improve(query, response, reflection_result["reflection"])
                best_response = response
            else:
                print("  ✅ Response meets quality standards")
                break
        
        return {
            "query": query,
            "final_response": best_response,
            "iterations": i + 1,
            "reflections": reflections
        }

# Create and test the self-reflecting agent
reflecting_agent = SelfReflectingAgent(llm)

# Test with a complex query
test_query = "What are the implications of quantum computing for cybersecurity?"

print("🤖 SELF-REFLECTING AGENT DEMO\n")
print("="*60 + "\n")

result = reflecting_agent.execute(test_query)

print("\n" + "="*60)
print("📊 FINAL RESULTS:\n")
print(f"Query: {result['query']}\n")
print(f"Iterations: {result['iterations']}\n")
print("Final Response:")
print("-"*40)
print(result['final_response'])
print("-"*40)

print("\n💡 This agent pattern combines:")
print("- Chain-of-Thought (research phase)")
print("- Role-based prompting (research assistant)")
print("- Self-reflection (quality evaluation)")
print("- Iterative improvement (refinement loop)")

## 🎓 Workshop Conclusion: Your Prompt Engineering Toolkit

### **What You've Mastered Today:** ✅

1. **Chain-of-Thought (CoT)** - Making AI think step-by-step
2. **Few-Shot Learning** - Teaching by example
3. **Role-Based Prompting** - Creating expert personas
4. **Robust Templates** - Production-ready prompt systems
5. **Debugging Techniques** - Fixing common failures
6. **Agentic Frameworks** - Building intelligent AI systems

### **Key Takeaways:** 💡

- **Specificity beats ambiguity** - Clear instructions = better results
- **Examples are powerful** - Show, don't just tell
- **Roles add expertise** - Context shapes responses
- **Iteration is key** - Test, reflect, improve
- **Combine techniques** - Mix methods for best results

### **Next Steps:** 🚀

1. **Practice** - Try these techniques on your own problems
2. **Experiment** - Combine different approaches
3. **Build** - Create your own AI agents
4. **Share** - Join the prompt engineering community
5. **Stay Updated** - This field evolves rapidly!

In [ ]:
# 🎯 Final Challenge: Build Your Own Custom Agent!

print("🏆 FINAL CHALLENGE: Create Your Dream AI Agent!\n")
print("="*60)
print("""
Combine everything you've learned to create a custom agent that:
1. Uses Chain-of-Thought for complex reasoning
2. Applies Few-Shot learning for consistent outputs
3. Adopts a specific Role/Persona
4. Has custom tools and capabilities
5. Can self-reflect and improve

Some ideas:
- 📚 A study buddy that helps learn new topics
- 🍳 A chef that creates recipes from ingredients
- 💼 A career coach for interview prep
- 🎮 A game master for D&D campaigns
- 📈 A financial advisor for investments
""")

# Template for your custom agent
class YourCustomAgent:
    """Replace this with your own agent implementation!"""
    
    def __init__(self, llm, role, tools=None):
        self.llm = llm
        self.role = role
        self.tools = tools or []
        self.memory = []
        
    def think(self, task):
        """Add Chain-of-Thought reasoning"""
        # Your CoT implementation
        pass
    
    def learn_from_examples(self, examples):
        """Add Few-Shot learning"""
        # Your few-shot implementation
        pass
    
    def act(self, user_input):
        """Main agent action"""
        # Combine all techniques here
        pass

# Space for your implementation
print("\n📝 Your turn to shine! Create your agent below:")
print("Uncomment and modify the template to build your dream AI assistant!")

# Example starter code
"""
my_agent = YourCustomAgent(
    llm=llm,
    role="Your chosen role",
    tools=["tool1", "tool2"]
)

# Test your agent
response = my_agent.act("Your test input")
print(response)
"""

print("\n" + "="*60)
print("🎉 Congratulations on completing the Advanced Prompt Engineering Workshop!")
print("You're now equipped to build powerful AI applications!")
print("\nRemember: The best way to learn is by doing. Start building! 🚀")

## 📖 Additional Resources & Next Steps

### **📚 Recommended Reading**
- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)
- [Anthropic's Prompt Engineering Guide](https://docs.anthropic.com/claude/docs/prompt-engineering)

### **🛠️ Tools & Libraries to Explore**
- **LangSmith**: Debug and monitor LangChain applications
- **Promptflow**: Microsoft's prompt engineering toolkit
- **Guidance**: Structured generation library
- **DSPy**: Declarative prompt programming

### **🎯 Practice Challenges**
1. Build a multi-lingual translator using few-shot examples
2. Create a code reviewer that uses CoT to explain issues
3. Design a creative writing assistant with multiple personas
4. Implement a fact-checker that self-reflects on accuracy

### **🌟 Community & Support**
- Join the [LangChain Discord](https://discord.gg/langchain)
- Follow [r/PromptEngineering](https://reddit.com/r/promptengineering)
- Share your work on GitHub with #promptengineering

### **📝 Workshop Feedback**
We'd love to hear about your experience! Consider:
- What techniques worked best for you?
- What challenges did you face?
- What would you like to learn next?

---

**Thank you for participating in this Advanced Prompt Engineering Workshop!** 🎉

Remember: The key to mastery is practice. Start small, experiment often, and don't be afraid to push boundaries. The future of AI interaction is in your hands!

Happy prompting! 🚀

## 📚 Best Practices & Pro Tips

### **🏆 Golden Rules of Prompt Engineering**

1. **Start Simple, Then Iterate**
   - Begin with a basic prompt
   - Test and identify weaknesses
   - Add complexity gradually

2. **Be Explicit About Format**
   - Specify output structure (JSON, bullet points, paragraphs)
   - Provide format examples
   - Use delimiters for clarity

3. **Context is King**
   - Provide relevant background
   - Define technical terms
   - Set clear boundaries

4. **Test Edge Cases**
   - Empty inputs
   - Very long inputs
   - Special characters
   - Contradictory instructions

5. **Version Control Your Prompts**
   - Track changes over time
   - Document what works
   - A/B test variations

### **⚡ Performance Optimization Tips**

- **Token Management**: Keep prompts concise to reduce costs
- **Caching**: Store common responses to avoid repeated API calls
- **Batch Processing**: Group similar requests together
- **Async Operations**: Use async/await for concurrent requests
- **Rate Limiting**: Implement backoff strategies

### **🛡️ Security Considerations**

- Never include sensitive data in prompts
- Validate and sanitize user inputs
- Implement prompt injection prevention
- Use environment variables for API keys
- Monitor for unusual usage patterns

In [ ]:
# 💼 Real-World Example: Production-Ready Prompt System

class ProductionPromptSystem:
    """A production-ready prompt engineering system with all best practices"""
    
    def __init__(self, llm, cache_enabled=True):
        self.llm = llm
        self.cache = {} if cache_enabled else None
        self.prompt_versions = {}
        self.metrics = {
            "total_requests": 0,
            "cache_hits": 0,
            "errors": 0,
            "avg_response_time": 0
        }
    
    def create_versioned_prompt(self, name: str, template: str, version: str = "1.0.0"):
        """Create and version a prompt template"""
        if name not in self.prompt_versions:
            self.prompt_versions[name] = {}
        
        self.prompt_versions[name][version] = PromptTemplate(
            input_variables=self._extract_variables(template),
            template=template
        )
        return f"Prompt '{name}' v{version} created"
    
    def _extract_variables(self, template: str) -> List[str]:
        """Extract variables from template string"""
        import re
        return list(set(re.findall(r'\{(\w+)\}', template)))
    
    def execute_with_fallback(self, prompt_name: str, inputs: Dict, 
                            version: str = None, fallback_model: str = "gpt-3.5-turbo"):
        """Execute prompt with automatic fallback on failure"""
        start_time = time.time()
        self.metrics["total_requests"] += 1
        
        # Check cache first
        cache_key = f"{prompt_name}:{str(inputs)}"
        if self.cache and cache_key in self.cache:
            self.metrics["cache_hits"] += 1
            return self.cache[cache_key]
        
        try:
            # Get the appropriate prompt version
            if version:
                prompt = self.prompt_versions[prompt_name][version]
            else:
                # Use latest version
                versions = list(self.prompt_versions[prompt_name].keys())
                prompt = self.prompt_versions[prompt_name][versions[-1]]
            
            # Execute with primary model
            chain = LLMChain(llm=self.llm, prompt=prompt)
            response = chain.run(**inputs)
            
            # Cache successful response
            if self.cache is not None:
                self.cache[cache_key] = response
            
        except Exception as e:
            print(f"⚠️ Primary model failed: {e}")
            self.metrics["errors"] += 1
            
            # Fallback to simpler model
            try:
                print(f"🔄 Falling back to {fallback_model}...")
                fallback_llm = ChatOpenAI(model=fallback_model, temperature=0.7)
                chain = LLMChain(llm=fallback_llm, prompt=prompt)
                response = chain.run(**inputs)
            except Exception as fallback_error:
                return f"Error: Both primary and fallback models failed - {fallback_error}"
        
        # Update metrics
        elapsed = time.time() - start_time
        self.metrics["avg_response_time"] = (
            (self.metrics["avg_response_time"] * (self.metrics["total_requests"] - 1) + elapsed) 
            / self.metrics["total_requests"]
        )
        
        return response
    
    def get_metrics(self) -> Dict:
        """Get system metrics"""
        return {
            **self.metrics,
            "cache_size": len(self.cache) if self.cache else 0,
            "prompt_versions": {k: list(v.keys()) for k, v in self.prompt_versions.items()}
        }

# Example usage
print("🏭 Production Prompt System Demo\n")

# Initialize the system
prod_system = ProductionPromptSystem(llm, cache_enabled=True)

# Create versioned prompts
prod_system.create_versioned_prompt(
    "analyzer",
    "Analyze this text and provide insights: {text}",
    "1.0.0"
)

prod_system.create_versioned_prompt(
    "analyzer",
    """Analyze this text with the following focus:
    
Text: {text}

Provide:
1. Main themes
2. Sentiment
3. Key takeaways

Format as bullet points.""",
    "2.0.0"
)

# Test the system
test_text = "Artificial intelligence is transforming how we work and live."

print("📊 Testing versioned prompts:\n")
response_v1 = prod_system.execute_with_fallback(
    "analyzer", 
    {"text": test_text},
    version="1.0.0"
)
print(f"Version 1.0.0 Response:\n{response_v1}\n")

response_v2 = prod_system.execute_with_fallback(
    "analyzer",
    {"text": test_text},
    version="2.0.0"
)
print(f"Version 2.0.0 Response:\n{response_v2}\n")

# Display metrics
print("\n📈 System Metrics:")
metrics = prod_system.get_metrics()
for key, value in metrics.items():
    print(f"  {key}: {value}")

## 🚀 Project Ideas: Build Your Own Agentic AI Systems!

### **🎯 Beginner Projects (Start Here!)**

#### 1. **📝 Smart Note-Taking Agent**
- **Goal**: Build an agent that captures, organizes, and retrieves notes intelligently
- **Features**:
  - Auto-categorize notes using few-shot learning
  - Summarize long notes with CoT
  - Search notes semantically
  - Generate study guides from notes
- **Tech Stack**: LangChain + Vector DB (Chroma/Pinecone)
- **Challenge**: Add voice-to-note transcription!

#### 2. **🎮 Interactive Story Game Master**
- **Goal**: Create a D&D-style game master that adapts to player choices
- **Features**:
  - Dynamic story generation with role-based prompting
  - Character personality management
  - Combat system with CoT calculations
  - World state tracking with memory
- **Tech Stack**: LangChain + ConversationBufferMemory
- **Challenge**: Add image generation for scenes!

#### 3. **🍳 Meal Planning Assistant**
- **Goal**: Agent that plans weekly meals based on preferences and budget
- **Features**:
  - Recipe generation from available ingredients (few-shot)
  - Nutritional analysis with CoT
  - Shopping list optimization
  - Dietary restriction handling
- **Tech Stack**: LangChain + Custom Tools
- **Challenge**: Integrate with grocery delivery APIs!

### **🔧 Intermediate Projects**

#### 4. **💻 Code Review & Refactoring Agent**
- **Goal**: Intelligent code analyzer that suggests improvements
- **Features**:
  - Multi-file code analysis with CoT
  - Pattern detection using few-shot examples
  - Security vulnerability scanning
  - Auto-generate unit tests
  - Refactoring suggestions with explanations
- **Tech Stack**: LangChain + AST parsing + GitHub API
- **Challenge**: Create PR comments automatically!

#### 5. **📊 Data Analysis Copilot**
- **Goal**: Agent that helps analyze datasets and generate insights
- **Features**:
  - Natural language to SQL/Pandas (few-shot)
  - Statistical analysis with CoT reasoning
  - Visualization recommendations
  - Anomaly detection and explanation
- **Tech Stack**: LangChain + Pandas + Plotly
- **Challenge**: Add predictive modeling suggestions!

#### 6. **🎓 Personalized Learning Tutor**
- **Goal**: Adaptive tutor that adjusts to student learning style
- **Features**:
  - Knowledge assessment with CoT
  - Personalized explanation generation (role-based)
  - Practice problem generation (few-shot)
  - Progress tracking and reporting
  - Multiple teaching strategies
- **Tech Stack**: LangChain + Knowledge Graph
- **Challenge**: Add spaced repetition scheduling!

### **🚀 Advanced Projects**

#### 7. **🤝 Multi-Agent Customer Support System**
- **Goal**: Team of specialized agents handling customer queries
- **Features**:
  - Router agent (classifies queries)
  - Technical support agent (CoT troubleshooting)
  - Billing agent (database integration)
  - Escalation agent (complex issues)
  - Supervisor agent (quality control)
- **Tech Stack**: LangChain + Multi-Agent Framework + CRM Integration
- **Challenge**: Add sentiment-based routing!

#### 8. **🔬 Research Assistant Network**
- **Goal**: Collaborative agents for academic research
- **Features**:
  - Literature review agent (summarization)
  - Hypothesis generator (creative CoT)
  - Experiment designer (methodological reasoning)
  - Data analyzer (statistical CoT)
  - Paper writer (structured generation)
- **Tech Stack**: LangChain + Arxiv API + Citation Management
- **Challenge**: Add peer review simulation!

#### 9. **🏢 Business Intelligence Dashboard**
- **Goal**: Executive assistant for business metrics and decisions
- **Features**:
  - KPI monitoring with explanations
  - Competitive analysis (web scraping + analysis)
  - Market trend prediction (CoT reasoning)
  - Report generation (template-based)
  - What-if scenario modeling
- **Tech Stack**: LangChain + BI Tools + Real-time Data Feeds
- **Challenge**: Add voice-activated queries!

### **🌟 Expert-Level Projects**

#### 10. **🧠 Self-Improving Documentation System**
- **Goal**: Agent that writes and maintains documentation autonomously
- **Features**:
  - Code-to-documentation generation
  - Documentation quality scoring
  - Self-reflection and improvement
  - Version-aware updates
  - Multi-language support
- **Implementation Tips**:
  ```python
  # Self-reflection loop
  1. Generate documentation
  2. Evaluate quality (separate prompt)
  3. Identify weaknesses
  4. Regenerate with improvements
  5. Compare versions and select best
  ```

#### 11. **🎯 Autonomous Project Manager**
- **Goal**: AI that manages software projects end-to-end
- **Features**:
  - Requirement analysis (CoT breakdown)
  - Task decomposition and estimation
  - Resource allocation optimization
  - Risk assessment and mitigation
  - Progress tracking and reporting
  - Stakeholder communication drafts
- **Tech Stack**: LangChain + Jira API + Slack + Calendar Integration

#### 12. **🔮 Predictive Debugging Assistant**
- **Goal**: Predict and prevent bugs before they happen
- **Features**:
  - Code pattern analysis for bug prediction
  - Test coverage gap identification
  - Performance bottleneck detection
  - Security vulnerability scanning
  - Automated fix suggestions with explanations
- **Tech Stack**: LangChain + Static Analysis Tools + ML Models

### **💡 Implementation Tips for All Projects**

#### **Architecture Best Practices:**
```python
# 1. Modular Agent Design
class BaseAgent:
    def __init__(self, llm, tools, memory):
        self.llm = llm
        self.tools = tools
        self.memory = memory
    
    def process(self, input):
        # Standard processing pipeline
        pass

# 2. Prompt Versioning
prompts = {
    "v1.0": "Basic prompt",
    "v1.1": "Improved with CoT",
    "v2.0": "Added few-shot examples"
}

# 3. Error Handling
try:
    response = agent.run(query)
except Exception as e:
    fallback_response = simple_agent.run(query)
    log_error(e)

# 4. Performance Monitoring
@track_performance
def agent_action(input):
    start = time.time()
    result = process(input)
    log_metrics(time.time() - start)
    return result
```

#### **Testing Strategies:**
- **Unit Testing**: Test individual prompts and tools
- **Integration Testing**: Test agent chains end-to-end
- **Regression Testing**: Ensure improvements don't break existing functionality
- **A/B Testing**: Compare different prompt strategies
- **User Testing**: Get feedback on agent responses

#### **Scaling Considerations:**
1. **Caching**: Store common responses to reduce API calls
2. **Async Processing**: Handle multiple requests concurrently
3. **Rate Limiting**: Implement backoff strategies
4. **Cost Optimization**: Use smaller models for simple tasks
5. **Monitoring**: Track token usage and response times

### **🎪 Fun Variations & Challenges**

#### **Make It Social:**
- Add personality traits to agents
- Create agent debates/discussions
- Build agent teaching agent scenarios
- Implement agent collaboration protocols

#### **Gamification Ideas:**
- Points system for agent performance
- Leaderboards for multi-agent systems
- Achievement unlocks for capabilities
- Evolution system (agents improve over time)

#### **Creative Twists:**
- **Time Traveler Agent**: Responds as if from different time periods
- **Multilingual Mediator**: Translates and cultural adapts
- **Dream Interpreter**: Analyzes and explains dreams with symbolism
- **Code Poetry Generator**: Turns code into poetry and vice versa
- **Conspiracy Theory Debunker**: Uses CoT to analyze claims

### **📚 Resources for Building**

#### **Frameworks & Libraries:**
- **LangChain**: Main framework for agent development
- **AutoGen**: Microsoft's multi-agent framework
- **CrewAI**: High-level agent collaboration
- **BabyAGI**: Autonomous task management
- **AutoGPT**: Fully autonomous agents

#### **Vector Databases:**
- **Pinecone**: Managed vector database
- **Chroma**: Open-source embeddings DB
- **Weaviate**: GraphQL-based vector search
- **Qdrant**: High-performance vector similarity

#### **Useful APIs:**
- **Serper**: Google search API for agents
- **Wolfram Alpha**: Computational knowledge
- **OpenWeather**: Weather data
- **NewsAPI**: News aggregation
- **GitHub API**: Code repository access

### **🏆 Success Metrics**

Track your agent's performance with:
1. **Response Accuracy**: How often is the agent correct?
2. **Task Completion Rate**: % of tasks successfully completed
3. **User Satisfaction**: Feedback scores
4. **Response Time**: Average processing duration
5. **Token Efficiency**: Output quality per token used
6. **Error Rate**: Frequency of failures
7. **Learning Curve**: Improvement over time

### **🚀 Next Steps**

1. **Pick a project** that excites you
2. **Start simple** - build an MVP first
3. **Iterate quickly** - test and improve
4. **Share your work** - get community feedback
5. **Combine projects** - create agent ecosystems!

Remember: The best agent is one that solves a real problem you care about! 🌟